In [26]:
# AK Protein Eigenvector Comparison: AF3 vs MD for ChimeraX
import numpy as np
import urllib.request
import os

print("🧬 AK Protein Dynamics Comparison: AF3 vs MD")
print("Generating porcupine plots for ChimeraX visualization")

🧬 AK Protein Dynamics Comparison: AF3 vs MD
Generating porcupine plots for ChimeraX visualization


In [27]:
# Flexible eigenvector parser for both AF3 and MD formats
def parse_eigenvectors_flexible(filename):
    """Parse eigenvector file - handles both AF3 and MD formats"""
    pcs = {}
    current_pc = None
    vectors = []
    
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('PC'):
                # Save previous PC if exists
                if current_pc is not None and vectors:
                    pcs[current_pc] = np.array(vectors)
                
                # Start new PC - handle different formats
                parts = line.split()
                current_pc = int(parts[0][2:])  # Extract PC number
                
                # Try different eigenvalue formats
                if '=' in line:
                    # AF3 format: PC1 eigenvalue=56.434360
                    eigenvalue = float(line.split('=')[1].replace('):', ''))
                elif '(' in line and 'eigenvalue' in line:
                    # MD format: PC1 atom displacements (eigenvalue=24139.882626270522):
                    eigenvalue_part = line.split('(eigenvalue=')[1]
                    eigenvalue = float(eigenvalue_part.replace('):', ''))
                else:
                    eigenvalue = 0.0  # Default if no eigenvalue found
                
                print(f"Found {parts[0]} with eigenvalue {eigenvalue}")
                vectors = []
            else:
                # Parse vector components (x, y, z)
                if line:  # Skip empty lines
                    components = [float(x) for x in line.split()]
                    vectors.append(components)
    
    # Save last PC
    if current_pc is not None and vectors:
        pcs[current_pc] = np.array(vectors)
    
    return pcs

# Load AF3 eigenvectors
print("Loading AF3 eigenvectors...")
ak_af_pcs = parse_eigenvectors_flexible("../data/ak_af_eigen.txt")
combined_af_ev = ak_af_pcs[1] + ak_af_pcs[2] + ak_af_pcs[3]
print(f"AF3 combined shape: {combined_af_ev.shape}")

# Load MD eigenvectors
print("\nLoading MD eigenvectors...")
ak_md_pcs = parse_eigenvectors_flexible("../data/ak_md_run1eigen.txt")
combined_md_ev = ak_md_pcs[1] + ak_md_pcs[2] + ak_md_pcs[3]
print(f"MD combined shape: {combined_md_ev.shape}")

Loading AF3 eigenvectors...
Found PC1 with eigenvalue 56.43436
Found PC2 with eigenvalue 2.61647
Found PC3 with eigenvalue 0.893096
Found PC4 with eigenvalue 0.475852
Found PC5 with eigenvalue 0.219268
Found PC6 with eigenvalue 0.16368
Found PC7 with eigenvalue 0.091615
Found PC8 with eigenvalue 0.077219
Found PC9 with eigenvalue 0.058031
Found PC10 with eigenvalue 0.053173
Found PC11 with eigenvalue 0.046774
AF3 combined shape: (214, 3)

Loading MD eigenvectors...
Found PC1 with eigenvalue 24139.882626270522
Found PC2 with eigenvalue 16267.34706505672
Found PC3 with eigenvalue 10558.477334999354
MD combined shape: (214, 3)


In [31]:
# Download PDB structure and extract coordinates
pdb_id = "4AKE"
pdb_file = f"{pdb_id.lower()}.pdb"

if not os.path.exists(pdb_file):
    print(f"Downloading PDB structure {pdb_id}...")
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    urllib.request.urlretrieve(url, pdb_file)
    print(f"Downloaded {pdb_file}")
else:
    print(f"{pdb_file} already exists")

def extract_ca_coordinates(pdb_file, target_chain='A'):
    """Extract CA atom coordinates from specific chain"""
    coords = []
    chain_ids = []
    residue_nums = []
    
    with open(pdb_file, 'r') as f:
        for line in f:
            if line.startswith('ATOM') and line[12:16].strip() == 'CA' and line[21:22].strip() == target_chain:
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                coords.append([x, y, z])
                chain_ids.append(target_chain)
                residue_nums.append(int(line[22:26].strip()))
    
    return np.array(coords), chain_ids, residue_nums

# Extract coordinates from chain A (matches eigenvector size: 214)
coords, chain_ids, residue_nums = extract_ca_coordinates(pdb_file, 'A')
print(f"Extracted {len(coords)} CA coordinates from chain A")

# Verify sizes match both AF3 and MD eigenvectorselse:

if len(coords) == combined_af_ev.shape[0] == combined_md_ev.shape[0]:
    print("✓ All datasets match: 214 CA atoms")

4ake.pdb already exists
Extracted 214 CA coordinates from chain A
✓ All datasets match: 214 CA atoms


In [32]:
# Final dual-vector BILD function (cone-based only)
def write_dual_vector_bild_cones(output_file, pdb_file, coords, vectors1, vectors2, 
                                scale=8.0, color1="red", color2="dark blue", 
                                label1="AF3", label2="MD"):
    """Write BILD format with two vector sets using cone arrowheads"""
    
    bild_file = output_file.replace('.cxc', '.bild')
    
    # Write BILD file with two vector sets
    with open(bild_file, 'w') as f:
        f.write(f"# BILD format for ChimeraX dual porcupine plot\n")
        f.write(f"# {label1} vectors in {color1}, {label2} vectors in {color2}\n")
        
        # First vector set (AF3)
        f.write(f".color {color1}\n")
        for coord, vec in zip(coords, vectors1):
            vec_scaled = vec * scale
            end_point = coord + vec_scaled
            
            # Draw cylinder for arrow shaft
            f.write(f".cylinder {coord[0]:.3f} {coord[1]:.3f} {coord[2]:.3f} ")
            f.write(f"{end_point[0]:.3f} {end_point[1]:.3f} {end_point[2]:.3f} 0.04\n")
            
            # Draw cone for arrowhead
            cone_base = coord + vec_scaled * 0.8
            f.write(f".cone {cone_base[0]:.3f} {cone_base[1]:.3f} {cone_base[2]:.3f} ")
            f.write(f"{end_point[0]:.3f} {end_point[1]:.3f} {end_point[2]:.3f} 0.12\n")
        
        # Second vector set (MD) - slight offset to distinguish
        f.write(f"\n.color {color2}\n")
        offset = 0.3  # Small offset to separate overlapping arrows
        for coord, vec in zip(coords, vectors2):
            vec_scaled = vec * scale
            # Apply small offset
            offset_coord = coord + np.array([offset, 0, 0])
            end_point = offset_coord + vec_scaled
            
            # Draw cylinder for arrow shaft
            f.write(f".cylinder {offset_coord[0]:.3f} {offset_coord[1]:.3f} {offset_coord[2]:.3f} ")
            f.write(f"{end_point[0]:.3f} {end_point[1]:.3f} {end_point[2]:.3f} 0.04\n")
            
            # Draw cone for arrowhead
            cone_base = offset_coord + vec_scaled * 0.8
            f.write(f".cone {cone_base[0]:.3f} {cone_base[1]:.3f} {cone_base[2]:.3f} ")
            f.write(f"{end_point[0]:.3f} {end_point[1]:.3f} {end_point[2]:.3f} 0.12\n")
    
    # Write ChimeraX script
    with open(output_file, 'w') as f:
        f.write(f"# ChimeraX dual porcupine plot: {label1} vs {label2}\n")
        f.write(f"# {label1} ({color1}) vs {label2} ({color2})\n\n")
        
        f.write(f"open {pdb_file}\n")
        f.write("cartoon\n")
        f.write("color bychain\n")
        f.write("hide atoms\n\n")
        
        f.write(f"open {bild_file}\n")
        f.write("view\n")
        f.write("lighting soft\n")
        f.write("set bgColor white\n")
        f.write(f"# Legend: {color1}={label1}, {color2}={label2}\n")
    
    return bild_file

print("✓ Dual-vector cone function loaded")

✓ Dual-vector cone function loaded


In [ ]:
# Generate final dual-vector cone comparison for ChimeraX
print("🎯 Generating dual-vector cone comparison...")

scale = 8.0  # Optimized scale for clear visualization
output_file = "ak_af3_vs_md_cones.cxc"

# Generate the cone-based dual comparison
bild_file = write_dual_vector_bild_cones(
    output_file, pdb_file, coords,
    combined_af_ev, combined_md_ev,
    scale=scale, color1="red", color2="blue", 
    label1="AF3", label2="MD"
)

print(f"✓ Created ChimeraX script: {output_file}")
print(f"✓ Created BILD data file: {bild_file}")

# Compare vector magnitudes
af3_magnitude = np.mean(np.linalg.norm(combined_af_ev, axis=1))
md_magnitude = np.mean(np.linalg.norm(combined_md_ev, axis=1))
ratio = md_magnitude / af3_magnitude

print(f"\n📊 Vector Analysis:")
print(f"  AF3 magnitude: {af3_magnitude:.4f}")
print(f"  MD magnitude:  {md_magnitude:.4f}")
print(f"  MD/AF3 ratio:  {ratio:.2f}x")

print(f"\n🚀 Usage in ChimeraX:")
print(f"  open {output_file}")
print(f"\n🎨 Color scheme: 🔴 Red=AF3, 🔵 Blue=MD")
print(f"Arrows show combined motion from PC1+PC2+PC3")

🎯 Generating dual-vector cone comparison...
✓ Created ChimeraX script: ak_af3_vs_md_cones.cxc
✓ Created BILD data file: ak_af3_vs_md_cones.bild

📊 Vector Analysis:
  AF3 magnitude: 0.0910
  MD magnitude:  0.1041
  MD/AF3 ratio:  1.14x

🚀 Usage in ChimeraX:
  open ak_af3_vs_md_cones.cxc

🎨 Color scheme: 🔴 Red=AF3, 🔵 Dark Blue=MD
Arrows show combined motion from PC1+PC2+PC3


In [ ]:
# 🎯 Process All Four Protein Targets
# Now let's extend this to handle all protein targets systematically

proteins = {
    "ak": {
        "pdb_id": "4AKE", 
        "chain": "A",
        "af_file": "../data/ak_af_eigen.txt",
        "md_file": "../data/ak_md_run1eigen.txt",
        "description": "Adenylate Kinase"
    },
    "lipa": {
        "pdb_id": "1EX9",
        "chain": "A", 
        "af_file": "../data/lipa_af_eigen.txt", 
        "md_file": "../data/lipa_md_run1eigen.txt",
        "description": "Bacterial Lipase A"
    },
    "hiv": {
        "pdb_id": "1HPV",
        "chain": "A",
        "af_file": "../data/hiv_af_eigen.txt",
        "md_file": "../data/hiv_md_run1_eigen.txt", 
        "description": "HIV-1 Protease"
    },
    "onc": {
        "pdb_id": "1ONC", 
        "chain": "A",
        "af_file": "../data/onc_af_eigen.txt",
        "md_file": "../data/onc_md_run1_eigen.txt",
        "description": "Onconase"
    }
}

print("🧬 Multi-Protein Porcupine Plot Generator")
print("=" * 50)
for key, info in proteins.items():
    print(f"{key.upper()}: {info['description']} ({info['pdb_id']})")

print(f"\n✅ Ready to process {len(proteins)} protein targets!")

In [ ]:
# 🚀 Generate Porcupine Plots for All Proteins
import os

for protein_key, protein_info in proteins.items():
    print(f"\n🎯 Processing {protein_key.upper()}: {protein_info['description']}")
    print("-" * 60)
    
    try:
        # Download PDB structure
        pdb_id = protein_info['pdb_id']
        pdb_file = f"{pdb_id.lower()}.pdb"
        
        if not os.path.exists(pdb_file):
            print(f"📥 Downloading PDB structure {pdb_id}...")
            url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
            urllib.request.urlretrieve(url, pdb_file)
            print(f"✓ Downloaded {pdb_file}")
        else:
            print(f"✓ {pdb_file} already exists")
            
        # Extract coordinates
        coords, chain_ids, residue_nums = extract_ca_coordinates(pdb_file, protein_info['chain'])
        print(f"✓ Extracted {len(coords)} CA coordinates from chain {protein_info['chain']}")
        
        # Load AF3 eigenvectors
        print(f"📊 Loading AF3 eigenvectors from {protein_info['af_file']}...")
        if os.path.exists(protein_info['af_file']):
            af_pcs = parse_eigenvectors_flexible(protein_info['af_file'])
            combined_af_ev = af_pcs[1] + af_pcs[2] + af_pcs[3]
            print(f"✓ AF3 combined shape: {combined_af_ev.shape}")
        else:
            print(f"❌ AF3 file not found: {protein_info['af_file']}")
            continue
            
        # Load MD eigenvectors  
        print(f"📊 Loading MD eigenvectors from {protein_info['md_file']}...")
        if os.path.exists(protein_info['md_file']):
            md_pcs = parse_eigenvectors_flexible(protein_info['md_file'])
            combined_md_ev = md_pcs[1] + md_pcs[2] + md_pcs[3]
            print(f"✓ MD combined shape: {combined_md_ev.shape}")
        else:
            print(f"❌ MD file not found: {protein_info['md_file']}")
            continue
            
        # Check size compatibility
        min_size = min(len(coords), len(combined_af_ev), len(combined_md_ev))
        if len(coords) != len(combined_af_ev) or len(coords) != len(combined_md_ev):
            print(f"⚠️  Size mismatch - PDB: {len(coords)}, AF3: {len(combined_af_ev)}, MD: {len(combined_md_ev)}")
            print(f"🔧 Truncating all to minimum size: {min_size}")
            coords_truncated = coords[:min_size]
            combined_af_ev = combined_af_ev[:min_size]
            combined_md_ev = combined_md_ev[:min_size]
        else:
            coords_truncated = coords
            print(f"✓ All datasets match: {len(coords)} atoms")
        
        # Generate porcupine plot
        output_file = f"{protein_key}_af3_vs_md_cones.cxc"
        bild_file = write_dual_vector_bild_cones(
            output_file, pdb_file, coords_truncated,
            combined_af_ev, combined_md_ev,
            scale=8.0, color1="red", color2="blue", 
            label1="AF3", label2="MD"
        )
        
        # Calculate magnitude comparison
        af3_magnitude = np.mean(np.linalg.norm(combined_af_ev, axis=1))
        md_magnitude = np.mean(np.linalg.norm(combined_md_ev, axis=1))
        ratio = md_magnitude / af3_magnitude
        
        print(f"✓ Created: {output_file}")
        print(f"✓ Data: {bild_file}")
        print(f"📈 MD/AF3 magnitude ratio: {ratio:.2f}x")
        
    except Exception as e:
        print(f"❌ Error processing {protein_key}: {str(e)}")
        continue

print(f"\n🎉 Multi-protein processing complete!")
print(f"📁 Generated cone visualization files for all available proteins")
print(f"🚀 Usage: open <protein>_af3_vs_md_cones.cxc in ChimeraX")